In [8]:
import pandas as pd

Vamos a cargar los datos poblacionales.


In [9]:
df = pd.read_csv("raw/poblacion.csv", sep=";")

df.head()

,ANO,CODIGO_LOCALIDAD,NOMBRE_LOCALIDAD,SEXO,EDAD,ORDEN_MCV,CURSODEVIDA,ORDEN_GRUPO_EDAD,GRUPOEDAD,POBLACION_7
0,2005,1,Usaquén,Hombres,0,1,Primera Infancia,1,0 a 4,2909
1,2005,1,Usaquén,Hombres,1,1,Primera Infancia,1,0 a 4,2954
2,2005,1,Usaquén,Hombres,2,1,Primera Infancia,1,0 a 4,2919
3,2005,1,Usaquén,Hombres,3,1,Primera Infancia,1,0 a 4,2989
4,2005,1,Usaquén,Hombres,4,1,Primera Infancia,1,0 a 4,3079


Ahora lo que vamos a hacer es preparar los datos para que se tengamos los datos poblacionales de las localidades en cada año con columnas de TOTAL_HOMBRES TOTAL_MUJERES TOTAL.

In [10]:
# Agrupamos por año, localidad y sexo
poblacion_sexo = (
    df.groupby(['ANO', 'CODIGO_LOCALIDAD', 'NOMBRE_LOCALIDAD', 'SEXO'], as_index=False)['POBLACION_7']
      .sum()
      .rename(columns={'POBLACION_7': 'POBLACION_TOTAL'})
)

tabla_final = poblacion_sexo.pivot_table(
    index=['ANO', 'CODIGO_LOCALIDAD', 'NOMBRE_LOCALIDAD'],
    columns='SEXO',
    values='POBLACION_TOTAL',
    fill_value=0
).reset_index()

tabla_final['TOTAL'] = tabla_final['Hombres'] + tabla_final['Mujeres']
tabla_final = tabla_final.sort_values(['ANO', 'CODIGO_LOCALIDAD'])
tabla_final.rename(columns={'ANO': 'ANIO', 'CODIGO_LOCALIDAD': 'ID_LOCALIDAD'}, inplace=True)

# Ahora solo vamos a dejar los años que nos interesan 2010-2023
tabla_final = tabla_final[(tabla_final['ANIO'] >= 2010) & (tabla_final['ANIO'] <= 2023)]

# Aseguramos los tipos int en las columnas de población
tabla_final['Hombres'] = tabla_final['Hombres'].astype(int)
tabla_final['Mujeres'] = tabla_final['Mujeres'].astype(int)
tabla_final['TOTAL'] = tabla_final['TOTAL'].astype(int)

tabla_final.to_csv("consolidados/poblacion_localidades1.csv", index=False)
tabla_final



SEXO,ANIO,ID_LOCALIDAD,NOMBRE_LOCALIDAD,Hombres,Mujeres,TOTAL
105,2010,0,Bogotá,3406262,3690624,7096886
106,2010,1,Usaquén,224356,264494,488850
107,2010,2,Chapinero,65872,76881,142753
108,2010,3,Santa Fe,49963,50214,100177
109,2010,4,San Cristóbal,188149,198861,387010
...,...,...,...,...,...,...
394,2023,16,Puente Aranda,123355,129142,252497
395,2023,17,La Candelaria,8857,8190,17047
396,2023,18,Rafael Uribe Uribe,187444,193567,381011
397,2023,19,Ciudad Bolívar,327765,339603,667368


In [11]:
tabla_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 105 to 398
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ANIO              294 non-null    int64 
 1   ID_LOCALIDAD      294 non-null    int64 
 2   NOMBRE_LOCALIDAD  294 non-null    object
 3   Hombres           294 non-null    int64 
 4   Mujeres           294 non-null    int64 
 5   TOTAL             294 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 16.1+ KB
